In [33]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

df = pd.read_csv('train.csv')


### Verificação e exploração inicial

In [ ]:
df.info() 
#Vejo algumas informações em relação as colunas

#### Verificando se existe algum valor nulo


In [ ]:
df.isnull().sum()
#Aqui verifico se existe algum valor nulo.

#### Mais explorações de dados

In [ ]:
df.sample(10)
#Vejo 10 linhas aleatórias

In [ ]:
df.describe()
#Aqui vejo algumas estatísticas

#### Análise da Distribuição da Energia das Músicas

Realizei uma exploração adicional dos dados, criando um histograma para visualizar a distribuição da energia das músicas. O gráfico mostra a frequência de músicas em diferentes níveis de energia.

In [ ]:
plt.hist(df['energy'], bins=30, color='blue', alpha=0.7)
plt.title('Distribuição da Energia das Músicas')
plt.xlabel('Energy')
plt.ylabel('Frequência')
plt.show()


### Formulação de hipoteses

1° Hipotese: será que existe uma corrrelação entre a energia da musica com a popularidade? 

A primeira hipótese levantada foi se existe uma correlação entre o nível de energia das músicas e sua popularidade. A expectativa inicial era de que músicas mais enérgicas, por serem mais animadas e frequentemente tocadas em festas ou eventos, seriam mais populares.

No entanto, a análise dos dados demonstrou que as músicas com menor nível de energia tendem a ser mais populares. Isso sugere que, embora as músicas energéticas sejam masi ouvidas em certos contextos, as músicas mais calmas ou menos agitadas podem ter maior gosto geral do público, possivelmente por serem mais usada para momentos de relaxamento ou consumo diário.

In [ ]:
popularity_by_energy = df.groupby('energy')['popularity_target'].mean().reset_index()

plt.figure(figsize=(12, 6))
plt.bar(popularity_by_energy['energy'], popularity_by_energy['popularity_target'], color='royalblue')

plt.title('Popularity by Energy', fontsize=16)
plt.xlabel('Energy', fontsize=14)
plt.ylabel('Average Popularity', fontsize=14)
plt.xticks(rotation=45, fontsize=12)

plt.xlim(0, 1)
plt.xticks([round(i * 0.1, 1) for i in range(11)], fontsize=12)

plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


2° Hipotese: Corrrelação entre o genero musical com a popularidade?

A hipótese analisada questiona se existe uma correlação entre o gênero musical e sua popularidade. Inicialmente, a expectativa era que gêneros como o funk, bastante popular no Brasil, estariam no topo da lista. No entanto, os dados não confirmaram essa hipótese.

Para melhor visualização e análise, foram selecionados os 20 gêneros mais populares com base nos dados disponíveis. O gráfico abaixo demonstra que a relação entre gênero e popularidade é mais complexa do que o esperado, sugerindo que outros fatores além da nacionalidade ou o sucesso em um país específico podem influenciar na popularidade global das músicas.

In [ ]:
popularity_by_genre = df.groupby('track_genre')['popularity_target'].mean().reset_index()

top_genres = popularity_by_genre.nlargest(20, 'popularity_target')

plt.figure(figsize=(12, 8))
plt.barh(top_genres['track_genre'], top_genres['popularity_target'], color='lightblue')

plt.title('Average Popularity by Track Genre', fontsize=16)
plt.xlabel('Average Popularity', fontsize=14)
plt.ylabel('Track Genre', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


3° Hipotese: Correlação entre conteudo explicito e popularidade


A terceira hipótese avaliada foi se músicas com conteúdo explícito possuem maior popularidade em relação às músicas sem esse tipo de conteúdo. A análise inicial indicou uma diferença em favor das músicas explícitas, que apresentam uma leve vantagem em termos de popularidade. No entanto, a diferença não é tão significativa quanto o esperado. Essa observação sugere que, embora o conteúdo explícito possa ser popular, ele não é um fator decisivo para a popularidade. 

In [ ]:
popularity_by_explicit = df.groupby('explicit')['popularity_target'].mean().reset_index()

plt.figure(figsize=(8, 5))
plt.bar(popularity_by_explicit['explicit'], popularity_by_explicit['popularity_target'], color='lightcoral')

plt.title('Average Popularity by Explicit Content', fontsize=16)
plt.xlabel('Explicit Content (0 = No, 1 = Yes)', fontsize=14)
plt.ylabel('Average Popularity', fontsize=14)
plt.xticks([0, 1], ['No', 'Yes'], fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()


Verificação de colunas categoricas e numericas

In [ ]:
numeric_df = df.select_dtypes(include=['number'])
print('Colunas numéricas:\n', numeric_df.columns)
#Verifica novamente quais são as colunas numéricas e categoricas
#Esse trecho de codigo foi recolocado para verificar se alguma coluna é categorica
categorical_df = df.select_dtypes(exclude=['number'])
print('\nColunas categóricas:\n', categorical_df.columns)

### Tratamento nas Colunas Categóricas

O **One-Hot Encoding** foi aplicado à coluna `track_genre`, transformando cada valor distinto dessa coluna em uma nova coluna binária. Esse método foi escolhido porque a variável `track_genre` não possui uma relação de ordem natural entre suas categorias, tornando o One-Hot Encoding uma escolha adequada para representar os diferentes gêneros musicais sem introduzir qualquer hierarquia entre eles.

Além disso, as colunas `track_id`, `album_name`, `artists` e `track_name` foram removidas, pois essas variáveis são  identificadores únicos ou strings que não tem relação direta com a popularidade da música. Manter essas colunas poderia aumentar a dimensionalidade do modelo sem agregar valor ao modleo preditivo. Tirar elas ajuda a focar nas variáveis que tem impacto real na previsão de popularidade, como características numéricas e categorias relevantes.



In [ ]:
df_encoded = pd.get_dummies(df, columns=['track_genre'], drop_first=True)
df_encoded = df_encoded.drop(["track_id", "album_name", "artists", "track_name"], axis='columns')
df = df_encoded

print(df.head())

### Iniciação da construção do modelo e seleção de features



A seleção das melhores features foi feita utilizando o algoritmo de **Random Forest**, que, além de ser um modelo para classificação, também oferece uma métrica interna de **importância das variáveis**. Essa métrica nos permite identificar quais são as variáveis mais relevantes para a previsão da popularidade das músicas.

### Processo de Seleção

1. **Treinamento do Modelo Inicial**:
   Inicialmente, o modelo **Random Forest** foi treinado com todas as features disponíveis no dataset. Durante esse processo, o modelo calcula o impacto de cada feature na qualidade das previsões, gerando uma pontuação de importância para cada uma delas.

2. **Cálculo da Importância das Features**:
   Após o treinamento do modelo, a importância das features foi extraída. Essa importância é baseada no quanto cada feature contribui para a divisão dos nós nas árvores de decisão que compõem a Random Forest.

3. **Seleção das 10 Features Mais Importantes**:
   Com base nas pontuações de importância, as 10 features com maior impacto no modelo foram selecionadas para a construção final. Essas variáveis foram consideradas mais relevantes para prever a popularidade das músicas.


In [ ]:

X = df.drop('popularity_target', axis=1)  
y = df['popularity_target'] 

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

model = RandomForestClassifier(random_state=42)

model.fit(X_train, y_train)

importances = model.feature_importances_
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Ordena o DataFrame pela importância
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

df = feature_importances

# Exibir as 10 features mais importantes
print("Top 10 Features Mais Importantes:")
print(feature_importances.head(10))


### Ajustes de Hiperparâmetro

Foi utilizado o **GridSearchCV** pois é uma técnica eficiente para otimizar os hiperparâmetros de modelos de aprendizado de máquina. Embora o processo de busca possa demorar um pouco, ele proporciona melhorias significativas no desempenho do modelo ao conseguir:

1. **Explorar Várias Combinações de Hiperparâmetros**: O GridSearch permite testar diferentes combinações de valores para os hiperparâmetros definidos, garantindo que o modelo encontre a melhor configuração possível.

2. **Selecionar Hiperparâmetros Específicos**: O GridSearch permite a personalização dos hiperparâmetros que você deseja otimizar, como o número de árvores em um modelo Random Forest (`n_estimators`) ou a profundidade máxima das árvores (`max_depth`), o que pode afetar diretamente a complexidade e o desempenho do modelo.

Assim, embora o GridSearch possa exigir mais tempo de computação, o investimento em tempo frequentemente resulta em um modelo mais robusto e eficiente.


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [ 150, 200],  # número de árvores
    'max_depth': [None, 10, 20],  # profundidade máxima
}


grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

grid_search.fit(X_train_top, y_train)


print("Melhores hiperparâmetros:", grid_search.best_params_)


best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test_top)

from sklearn.metrics import accuracy_score, classification_report


print("Acurácia:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


### Iniciando o modelo com Hiperparâmetros ajustados e Colunas selecionadas

O inncio do treinamento do modelo **Random Forest** se deu com hiperparâmetros ajustados. Ajustando os parâmetros como o número de árvores (`n_estimators`) e a profundidade máxima (`max_depth`) para melhorar o desempenho do modelo.

Além disso, está sendo ultilizado apenas as colunas mais relevantes, baseadas na importância das variáveis, garantindo que o modelo se concentre nas features que realmente impactam a previsão da popularidade das músicas.


In [ ]:
top_features = [feature for feature in feature_importances['Feature'].head(10) if feature in X_train.columns]

X_train_top = X_train[top_features]
X_test_top = X_test[top_features]

best_model = RandomForestClassifier(max_depth=None, n_estimators=200, random_state=42)
best_model.fit(X_train_top, y_train)

y_pred = best_model.predict(X_test_top)

print("Acurácia:", accuracy_score(y_test, y_pred))
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))


### Previsões com o Conjunto de Teste

O código realiza previsões sobre a popularidade das músicas usando um conjunto de dados carregado do arquivo `test.csv`. 

Primeiro, as colunas relevantes são selecionadas para criar um DataFrame filtrado (`test_df_filtered`). Em seguida, o modelo treinado (`best_model`) é utilizado para fazer previsões, e os resultados são armazenados em um novo DataFrame (`output_df`). Por fim, as previsões são salvas em um arquivo CSV chamado `sample_submission.csv`.


In [ ]:
import pandas as pd

test_df = pd.read_csv('test.csv')

print("Colunas do test.csv após transformação:", test_df.columns)

top_features = ['track_unique_id', 'acousticness', 'energy', 'danceability', 
                'valence', 'duration_ms', 'speechiness', 'loudness', 
                'liveness', 'tempo']

test_df_filtered = test_df[top_features]

test_df_filtered.head(15)

y_pred_new = best_model.predict(test_df_filtered)

output_df = pd.DataFrame({
    'track_unique_id': test_df['track_unique_id'],
    'popularity_target': y_pred_new
})

output_df.to_csv('sample_submission.csv', index=False)

print("Arquivo 'sample_submission.csv' foi salvo com sucesso!")
